# Background

Recall from the first homework the dataset from "X-RAY STAR CLUSTERS IN THE CARINA COMPLEX" Feigelson et al.

Their methodology works as follows:
1. They first use a density estimator to get a function $f : \mathbb{R^2} \to \mathbb{R}$.
2. Then, they consider the suplevel sets of $f$. That is, they find a suitable density threshold $\lambda > 0 \in \mathbb{R}$ and the set $S_\lambda = \{x \in \mathbb{R}^2 : f(x) \geq \lambda\}$.
3. Finally, they take as clusters the connected components of $S_\lambda$.

The choice of $\lambda$ is explained and justified in the paper.

# Exercise

1. Write down a function in Python that, given a point cloud $X \subseteq \mathbb{R}^2$, it clusters it using the procedure above.
To do this, you can use the functions defined below.
2. Describe the inputs and output of your procedure.
Can you take as input just a point cloud or do you require extra parameters?
3. Use that function to try to get a clustering similar to that of the paper mentioned above.
4. Can you think of a topological summary similar to that of Persistable that would be useful for choosing the parameters of your function?

In [ ]:
# basic imports and plotting functionality

import numpy as np
from sklearn import datasets

from matplotlib import pyplot as plt

# this library gives an easy way to create color palettes with
# very different colors. This is useful for coloring clusterings.
import glasbey

# a simple wrapper for glasbey
def cluster_colors(n_classes):
    return np.array(glasbey.create_palette(palette_size=n_classes))

# a function that will plot a 2D dataset and, optionally, color it according
# to a user given clustering. If no clustering is given, all data points are
# colored with the same color.
# Moreover, the cluster label is allowed to be -1. In that case the point is
# considered "unclustered" and is colored in gray
def plot_clustering(X, clustering = None, limits=None, sizes=[4, 2], noise_points=True, axis=True):
    if clustering is None:
        clustering = np.ones(X.shape[0], dtype=int)
    X_clustered = X[clustering != -1]
    X_unclustered = X[clustering == -1]
    clustering_clustered = clustering[clustering != -1]
    if noise_points:
        plt.scatter(
            X_unclustered[:, 0],
            X_unclustered[:, 1],
            s=sizes[1],
            c="grey",
        )
    plt.scatter(
        X_clustered[:, 0],
        X_clustered[:, 1],
        s=sizes[0],
        c=cluster_colors(max(clustering) + 1)[clustering_clustered],
    )
    if limits is not None:
        plt.xlim([limits[0],limits[1]])
        plt.ylim([limits[2],limits[3]])
    if not axis:
        plt.axis("Off")

In [ ]:
# compute connected components of graph and kernel density estimator

from scipy.sparse.csgraph import connected_components
from scipy.sparse import csr_matrix
from scipy.stats import gaussian_kde

def graph_connected_components(n_vertices, edges):
    """ Label vertices of a graph by their connected component.
    
    n_vertices: int
        The number of vertices in the graph. Vertices will be 
        {0, 1, 2, ..., n_vertices-1}.
    
    edges: list(pair(int,int))
        Each pair (i,j) in the list denotes an edge between i and j.
    """
    edge_weights = np.full(len(edges), 1, dtype=int)
    edges = np.array(edges)
    graph = csr_matrix((edge_weights, (edges[:,0], edges[:,1])), shape=(n_vertices, n_vertices))
    _ , labels = connected_components(csgraph=graph, directed=False, return_labels=True)
    return labels

def compute_kde(point_cloud, bandwidth):
    """ Compute a Gaussian kernel density estimator.
     
    point_cloud: ndarray
        An numpy array consisting of points in some Euclidean space.

    bandwidth: float
        The bandwidth used to compute the KDE
    """
    return gaussian_kde(np.vstack((point_cloud[:,0],point_cloud[:,1])), bandwidth)
    
def plot_suplevelset(values_on_grid, xmin, xmax, ymin, ymax, grid_granularity_x, grid_granularity_y, contour=False):
    xx, yy = np.mgrid[xmin:xmax:grid_granularity_x*1j, ymin:ymax:grid_granularity_y*1j]
    fig = plt.figure()
    ax = fig.gca()
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)
    if not contour:
        ax.imshow(np.rot90(values_on_grid), cmap='Blues', extent=[xmin, xmax, ymin, ymax])
    else:
        cset = ax.contour(xx, yy, values_on_grid, colors='k')
        ax.clabel(cset, inline=1, fontsize=10)

### Examples using the functions above

In [ ]:
# compute the connected components of a graph with 4 vertices and 2 edges
n_vertices = 4
edges = [[0,1],[1,2]]
print(graph_connected_components(n_vertices, edges))


In [ ]:
# compute a Gaussian KDE and plot it
data_blobs = datasets.make_blobs(n_samples=500, n_features=2, random_state=10)[0]
plot_clustering(data_blobs)
kde = compute_kde(data_blobs, 0.1)

xmin = -3
xmax = 8
ymin = -12
ymax = 8
grid_granularity_x = 100
grid_granularity_y = 100

xx, yy = np.mgrid[xmin:xmax:grid_granularity_x*1j, ymin:ymax:grid_granularity_y*1j]
grid = np.vstack([xx.ravel(), yy.ravel()])
f = np.reshape(kde(grid).T, xx.shape)

plot_suplevelset(f, xmin, xmax, ymin, ymax, grid_granularity_x, grid_granularity_y, contour=True)


# A warm up exercise

First use your function to cluster the dataset `data_blobs`, defined right above.

In [ ]:
# ...

# Dataset from paper

In [ ]:
# download table3 from https://cdsarc.cds.unistra.fr/ftp/J/ApJS/194/9/
# this contains the main dataset of
# "X-RAY STAR CLUSTERS IN THE CARINA COMPLEX" Feigelson et al.
import requests
url = "https://cdsarc.cds.unistra.fr/ftp/J/ApJS/194/9/table3.dat"
response = requests.get(url)
data = response.text

# we now preprocess the data we downloaded. We will consider the subsample
# referred to as the "spatially complete sample" in section 2 of the paper.
# See https://cdsarc.cds.unistra.fr/ftp/J/ApJS/194/9/ReadMe for a description
# of the dataset
unfiltered_dataset = []
large_scale_region = []
class_broos = []
cluster_feigelson = []
photon_flux = []
for line in data.split('\n'):
    row = line.split()
    if len(row) > 0:
        # these are the two spatial coordinates of the data points
        unfiltered_dataset.append(row[3:5])
        # this quantifies how bright the source is
        photon_flux.append(row[6])
        # the following contains the label H2 for points that
        # are deemed to probably belong to the Carina complex
        class_broos.append(row[8])
        # this contains the cluster label as reported in the paper
        if len(row) == 11 and row[9][0] == "C":
            cluster_feigelson.append(int(row[9][1:]))
        else:
            cluster_feigelson.append(-1)
unfiltered_dataset = np.array(unfiltered_dataset, dtype=float)
cluster_feigelson = np.array(cluster_feigelson, dtype=int)
class_broos = np.array(class_broos, dtype=str)
photon_flux = np.array(photon_flux, dtype=float)

# photon threshold as in the paper
photon_flux_threshold = -5.9
bright_enough = (photon_flux > photon_flux_threshold)

# probable Carina members
probable_carina_members = (class_broos == "H2")

# we put the two conditions together
condition = bright_enough & probable_carina_members

# we filter the point cloud and the cluster labels using
# the two conditions above. So we are only looking for bright sources that
# are considered to probably belong to the Carina complex
X = unfiltered_dataset[condition]
cluster = cluster_feigelson[condition]
# print the size of the filtered data.
# NOTE: we are getting 2870 samples instead of the 3220 samples reported in
# section 2 of the paper. I do not know why this is. If you are interested,
# please look into it!
X.shape

In [ ]:
# clustering reported in the paper.
plt.figure(figsize=(5,7))
plot_clustering(X, cluster, sizes=[6,1])